In [39]:
import pandas as pd

In [40]:
df = pd.read_csv('Nba_player_data_2013.csv')

In [41]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [42]:
df['spread'] = df['team_score']-df['opp_score']

In [43]:
groups= df.groupby(['Game_Id','Team'],sort=False).sum()

In [44]:
matrix = groups.as_matrix()

In [45]:
ids = []
for game in df['Game_Id'].unique():
    ids.append(game+ '%aa'+game.split('_')[1])
    ids.append(game+ '%aa'+game.split('_')[2])

In [46]:
new_frame = pd.DataFrame(matrix, columns = groups.columns, index = ids)

In [47]:
team_data = new_frame[['3P','3PA', 'AST', 'BLK', 'FG', 'FGA', 'FT', 'FTA', 'ORB',
       'PF', 'PTS', 'STL', 'TOV', 'TRB','is_Home','Team_Win']]

In [48]:
team_data = team_data.reset_index()

In [49]:
team_data['Team'] = team_data['index'].apply(lambda x: x.split('%aa')[1])

In [50]:
team_data['index'] = team_data['index'].apply(lambda x: x.split('%aa')[0])

In [51]:
team_data['is_Home'] = team_data['is_Home'].apply(lambda x: x>0).astype(int)
team_data['Team_Win'] = team_data['Team_Win'].apply(lambda x: x>0).astype(int)

In [52]:
both_teams_total = team_data.groupby('index',sort=False).sum().reset_index()
##this gets all the opponents data on the same line
weird = pd.merge(both_teams_total,team_data,on='index')

In [53]:
#fill in opponent data in an insufficient way
team_data['opp_TRB'] = weird['TRB_x']-team_data['TRB']
team_data['opp_3P'] = weird['3P_x']-team_data['3P']
team_data['opp_3PA'] = weird['3PA_x']-team_data['3PA']
team_data['opp_AST'] = weird['AST_x']-team_data['AST']
team_data['opp_BLK'] = weird['BLK_x']-team_data['BLK']
team_data['opp_FG'] = weird['FG_x']-team_data['FG']
team_data['opp_FGA'] = weird['FGA_x']-team_data['FGA']
team_data['opp_FT'] = weird['FT_x']-team_data['FT']
team_data['opp_FTA'] = weird['FTA_x']-team_data['FTA']
team_data['opp_PTS'] = weird['PTS_x']-team_data['PTS']
team_data['opp_ORB'] = weird['ORB_x']-team_data['ORB']
team_data['opp_STL'] = weird['STL_x']-team_data['STL']
team_data['opp_PF'] = weird['PF_x']-team_data['PF']
team_data['opp_TOV'] = weird['TOV_x']-team_data['TOV']

In [54]:
def opponent(row):
    if row['Team'] == row['index'].split('_')[1]:
        return row['index'].split('_')[2]
    else:
        return row['index'].split('_')[1]

team_data['opponent'] =team_data.apply(opponent, axis=1)

In [55]:
team_data['Game_Id'] = team_data['index']
team_data.drop('index',axis=1, inplace=True)

In [56]:
##get rid of games that are playoffs

team_data.to_csv('team_boxscores_2013.csv')

In [57]:
team_data.head()

,3P,3PA,AST,BLK,FG,FGA,FT,FTA,ORB,PF,...,opp_FGA,opp_FT,opp_FTA,opp_PTS,opp_ORB,opp_STL,opp_PF,opp_TOV,opponent,Game_Id
0,9.0,19.0,17.0,6.0,36.0,93.0,6.0,10.0,13.0,26.0,...,71.0,22.0,32.0,97.0,10.0,4.0,13.0,20.0,Orlando,10/29/2013_Indiana_Orlando
1,7.0,17.0,17.0,18.0,34.0,71.0,22.0,32.0,10.0,13.0,...,93.0,6.0,10.0,87.0,13.0,10.0,26.0,17.0,Indiana,10/29/2013_Indiana_Orlando
2,8.0,21.0,27.0,4.0,41.0,83.0,13.0,23.0,10.0,21.0,...,93.0,18.0,28.0,116.0,18.0,8.0,23.0,19.0,LA Clippers,10/29/2013_LA Lakers_LA Clippers
3,14.0,29.0,23.0,6.0,42.0,93.0,18.0,28.0,18.0,23.0,...,83.0,13.0,23.0,103.0,10.0,11.0,21.0,16.0,LA Lakers,10/29/2013_LA Lakers_LA Clippers
4,7.0,26.0,23.0,4.0,35.0,83.0,18.0,23.0,11.0,27.0,...,72.0,22.0,29.0,107.0,5.0,10.0,21.0,18.0,Chicago,10/29/2013_Miami_Chicago
